In [1]:
from phasic_tonic.detect_phasic import detect_phasic, detect_phasic_v2
from phasic_tonic.PhasicTonic import PhasicTonic
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.runtime_logger import logger_setup
from phasic_tonic.utils import get_sequences, get_segments, load_data, get_start_end, preprocess

import math
import numpy as np
import pandas as pd
import pynapple as nap

from pathlib import Path
from tqdm.auto import tqdm
from scipy.io import loadmat

fs = 500
fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

logger = logger_setup()

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"
DATASET_DIR = "/home/nero/datasets/preprocessed"
OUTPUT_DIR1 = "/home/nero/phasic_tonic/data/analysis_output/whole_posttrial5/"
OUTPUT_DIR2 = "/home/nero/phasic_tonic/data/analysis_output/segmented_posttrial5/"

def half_round_up(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

compressed_datasets = list(Path(DATASET_DIR).glob('*.npz'))

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()

len(compressed_datasets)

699

# Whole post-trial 5

In [58]:
container = []
from phasic_tonic.PhasicTonic import PhasicTonic

with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, hpc_fname, _ = mapped_datasets[name]
        logger.debug("Loading: {0}".format(name))

        if metadata["treatment"] == 0 or metadata["treatment"] == 1:
            n_down = n_down_cbd
        elif metadata["treatment"] == 2 or metadata["treatment"] == 3:
            n_down = n_down_rgs
        elif metadata["treatment"] == 4:
            n_down = n_down_os
        
        # Load the LFP data
        lfpHPC = loadmat(hpc_fname)['HPC'].flatten()
        # Load the sleep states
        hypno = loadmat(states_fname)['states'].flatten()
        
        # Skip if no REM epoch is detected
        if(not (np.any(hypno == 5))):
            logger.debug("No REM detected. Skipping.")
            continue
        elif(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))) < 10):
            logger.debug("No REM longer than 10s. Skipping.")
            continue
        
        # Detect phasic intervals
        lfpHPC_down = preprocess(lfpHPC, n_down)
        phREM = detect_phasic(lfpHPC_down, hypno, targetFs)

        # Classify each epoch as tonic or phasic event
        for rem_idx in phREM:
            # Container for storing results and metadata
            data = metadata.copy()
            
            rem_start, rem_end = rem_idx[0], rem_idx[1]
            data["rem_start"] = rem_start
            data["rem_end"] = rem_end

            phasic_idx = phREM[rem_idx]

            result = []

            if phasic_idx:
                # Check if REM epoch starts with tonic state within 200ms
                if np.abs(rem_start*targetFs - phasic_idx[0][0])>0.2*targetFs:
                    result.append("T")

                for ph in phasic_idx:
                    result.append("P")
                    result.append("T")
                
                # Check if REM epoch ends with phasic state within 200ms 
                if np.abs(rem_end*targetFs - phasic_idx[-1][-1])<0.2*targetFs:
                    result[-2:] = "P"
            else:
                result = "T"

            data["event_strings"] = "".join(result)
            container.append(data)
            
df = pd.DataFrame(container)
df

  0%|          | 0/539 [00:00<?, ?it/s]

,rat_id,study_day,condition,treatment,trial_num,rem_start,rem_end,event_strings
0,5,8,HC,0,3,1331,1485,TPTPTPT
1,5,8,HC,0,3,2428,2475,TPT
2,5,8,HC,0,2,411,496,TPTPTPTPT
3,5,8,HC,0,2,1564,1620,TPTPTPT
4,5,8,HC,0,2,1625,1709,TPT
...,...,...,...,...,...,...,...,...
2120,11,4,OR,4,3,508,518,T
2121,11,4,OR,4,3,736,900,TPTPTPT
2122,11,4,OR,4,3,928,1012,TPT
2123,11,4,OR,4,3,1791,1928,TPTPTPTPTPT


In [59]:
df.to_csv(OUTPUT_DIR1+"phasic_tonic_strings_epoch.csv")

# Segmented post-trial 5

In [64]:
container = []

with tqdm(compressed_datasets) as datasets:
    for fname in datasets:
        metaname = str(fname.stem)

        datasets.set_postfix_str(metaname)
        metadata = get_metadata(metaname)

        rem_epochs = load_data(fname)
        print(f"Loaded:{metaname}")

        if not rem_epochs:
            continue

        phREM = detect_phasic_v2(rem_epochs, fs=500)
        
        # Classify each epoch as tonic or phasic event
        for rem_idx in phREM:
            # Container for storing results and metadata
            data = metadata.copy()
            
            rem_start, rem_end = rem_idx[0], rem_idx[1]
            data["rem_start"] = rem_start
            data["rem_end"] = rem_end

            phasic_idx = phREM[rem_idx]
            result = []

            if phasic_idx:
                # Check if REM epoch starts with tonic state within 200ms
                if np.abs(rem_start*targetFs - phasic_idx[0][0])>0.2*targetFs:
                    result.append("T")

                for ph in phasic_idx:
                    result.append("P")
                    result.append("T")
                
                # Check if REM epoch ends with phasic state within 200ms 
                if np.abs(rem_end*targetFs - phasic_idx[-1][-1])<0.2*targetFs:
                    result[-2:] = "P"
            else:
                result = "T"

            data["event_strings"] = "".join(result)
            container.append(data)

df = pd.DataFrame(container)
df

  0%|          | 0/699 [00:00<?, ?it/s]

Loaded:Rat2_SD2_OR_2_posttrial5-3
Loaded:Rat4_SD4_HC_3_posttrial4
Loaded:Rat5_SD3_OD_0_posttrial5-1
Loaded:Rat3_SD4_OD_4_posttrial2
Loaded:Rat9_SD16_OR-N_4_posttrial4
Loaded:Rat4_SD2_HC_0_posttrial2
Loaded:Rat4_SD4_OR-N_4_posttrial5-2
Loaded:Rat6_SD3_HC_4_posttrial1
Loaded:Rat4_SD4_OR-N_4_posttrial4
Loaded:Rat4_SD1_OR_0_posttrial5-3
Loaded:Rat3_SD3_OR_3_posttrial5-2
Loaded:Rat6_SD3_CON_2_posttrial5-2
Loaded:Rat9_SD2_CON_4_posttrial5-0
Loaded:Rat5_SD6_HC_1_posttrial2
Loaded:Rat9_SD2_CON_4_posttrial3
Loaded:Rat9_SD4_CON_2_posttrial3
Loaded:Rat3_SD3_OR_3_posttrial4
Loaded:Rat2_SD3_CON_2_posttrial5-2
Loaded:Rat4_SD5_OD_3_posttrial4
Loaded:Rat8_SD1_HC_3_posttrial4
Loaded:Rat1_SD1_OD_4_posttrial2
Loaded:Rat9_SD4_CON_2_posttrial5-1
Loaded:Rat13_SD3_OR_4_posttrial2
Loaded:Rat1_SD2_OD_2_posttrial4
Loaded:Rat9_SD10_OR_4_posttrial4
Loaded:Rat2_SD2_OR_2_posttrial5-1
Loaded:Rat4_SD16_OR_0_posttrial5-3
Loaded:Rat3_SD3_OR_0_posttrial3
Loaded:Rat11_SD2_OD_4_posttrial1
Loaded:Rat9_SD3_OR_2_posttrial5-3

,rat_id,study_day,condition,treatment,trial_num,rem_start,rem_end,event_strings
0,2,2,OR,2,5-3,9121,9168,TPT
1,2,2,OR,2,5-3,9430,9468,T
2,2,2,OR,2,5-3,9738,9820,TPTPTPT
3,4,4,HC,3,4,2396,2400,T
4,4,4,HC,3,4,2411,2419,T
...,...,...,...,...,...,...,...,...
2122,1,1,OD,4,5-2,7379,7428,T
2123,1,1,OD,4,5-2,7850,7866,T
2124,1,1,OD,4,5-2,8002,8034,TPT
2125,5,3,OD,0,5-0,1866,1943,TPTPT


In [ ]:
df

In [65]:
df.to_csv(OUTPUT_DIR2+"phasic_tonic_strings_epoch.csv")